# Setup

## Packages

In [2]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import regex as re
from nltk.tokenize import word_tokenize
from google.colab import drive
import os
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader,RandomSampler, SequentialSampler, Dataset
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
import torch.nn.functional as F

import transformers
from transformers import AutoTokenizer, AdamW, XLNetTokenizer, XLNetForSequenceClassification, get_linear_schedule_with_warmup, XLNetModel

from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
drive.mount('/content/drive')
FOLDERNAME = 'ColabNotebooks/263/263 Final Project/Data'
%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/ColabNotebooks/263/263 Final Project/Data


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [6]:
train = pd.read_csv("train_df.csv")
dev = pd.read_csv("evaluation_df.csv")

# Drop rows with NaN values in 'text1' or 'text2'
train = train.dropna()
dev = dev.dropna()

print(len(train), len(dev))

4842 4902


In [7]:
def rescale_inverted(old_value, old_min=0, old_max=3, new_min=1, new_max=-1):
    new_value = ((old_value - old_min) / (old_max - old_min)) * (new_min - new_max) + new_max
    return new_value

In [ ]:
# Calculate the weighted gold similarity.
for column in ['Geography','Entities','Time','Narrative','Overall','Style','Tone']:
    train[column] = train[column] - 1
for column in ['GEO',	'ENT',	'TIME',	'NAR',	'Overall',	'STYLE',	'TONE']:
    dev[column] = dev[column] - 1

scores_sum = ['Geography','Entities','Time','Narrative','Overall','Style','Tone']
train['weighted_gold_similarity'] = train[scores_sum].sum(axis=1) / 7

scores_sum = ['GEO',	'ENT',	'TIME',	'NAR',	'Overall',	'STYLE',	'TONE']
dev['weighted_gold_similarity'] = dev[scores_sum].sum(axis=1) / 7

train['Overall'] = train['Overall'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))
dev['Overall'] = dev['Overall'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))

train['weighted_gold_similarity'] = train['weighted_gold_similarity'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))
dev['weighted_gold_similarity'] = dev['weighted_gold_similarity'].apply(lambda x: rescale_inverted(x, old_min=0, old_max=3, new_min=-1, new_max=1))

<ipython-input-7-f560f308a6dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column] = train[column] - 1
<ipython-input-7-f560f308a6dd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['weighted_gold_similarity'] = train[scores_sum].sum(axis=1) / 7
<ipython-input-7-f560f308a6dd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [8]:
train = train[0:1000].dropna()
dev = dev.dropna()

In [ ]:
# split into train and development.
#Train, Val= train_test_split(train[0:500], test_size=0.2)

# Reset the indices
#Train = Train.reset_index(drop=True)
#Val = Val.reset_index(drop=True)

# Train

## Encode

In [ ]:
# Load XLNet base model and tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [ ]:
# Create a PyTorch DataLoader
class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Assume we have some training data in `train_articles` and `train_labels`
train_articles = [' '.join(pair) for pair in zip(train['text1'], train['text2'])]
train_encodings = tokenizer(train_articles, truncation=True, padding=True, max_length=512)

# Prepare the inputs for the DataLoader
input_ids = torch.tensor(train_encodings['input_ids'])
attention_mask = torch.tensor(train_encodings['attention_mask'])

# And a DataLoader
train_loader = DataLoader(TensorDataset(input_ids, attention_mask), batch_size=32, shuffle=True)


## Pearson Score

In [ ]:
## Load Model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=4)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
## Use model for prediction (without tuning)
model.eval()  # Set model to evaluation mode
predictions = []

with torch.no_grad():
    for batch in train_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Get the predicted class indices
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())  # Add to our prediction list


In [ ]:
train['xlnet_pred'] = predictions

In [ ]:
# Calculate Pearson correlation
correlation, _ = pearsonr(train['xlnet_pred'], train['Overall'])

# Save the correlation in the DataFrame
train['xlnet_correlation'] = correlation

In [11]:
train = pd.read_csv('train_similarities.csv')

In [12]:
np.mean(train['xlnet_correlation'])

0.06416689558583201

In [ ]:
train.to_csv('train_similarities.csv', index=False)

# Dev

In [ ]:
# Assume we have some training data in `train_articles` and `train_labels`
dev_articles = [' '.join(pair) for pair in zip(dev['text1'], dev['text2'])]
dev_encodings = tokenizer(dev_articles, truncation=True, padding=True, max_length=512)

In [ ]:
dev_dataset = ArticleDataset(dev_encodings)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=True)